# Recursive QAOA

In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import networkx as nx
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import random
import json
import os
import joblib

from openqaoa.qaoa_parameters import QAOACircuitParams, create_qaoa_variational_params, Hamiltonian
from openqaoa.devices import create_device
from openqaoa.backends.qaoa_backend import get_qaoa_backend
from openqaoa.optimizers import get_optimizer
from openqaoa.utilities import *
from openqaoa.workflows.optimizer import QAOA, RQAOA
from openqaoa.problems.problem import QUBO, MaximumCut, MinimumVertexCover

In [2]:
# Set plotting style
plt.rcParams["figure.figsize"] = (15,10)
plt.rcParams['figure.dpi'] = 100
plt.rcParams['savefig.dpi'] = 600
#plt.rcParams['font.size'] = '12'
#plt.rc('font', family='serif', size=600)

sns.axes_style("whitegrid")
sns.set_context("paper", font_scale=2.5)

In [3]:
def random_k_regular_graph_normal_distribution(degree: int,
                           nodes: List[int],
                           seed: int = None,
                           weighted: bool = False,
                           sigma: float = 1.0,
                           mu: float = 0.0,
                           biases: bool = False) -> nx.Graph:
    """
    Produces a random graph with specified number of nodes, each having degree k.

    Parameters
    ----------
    degree: `int`
        Desired degree for the nodes.
    nodes: `list`
        The node set of the graph.
    seed: `int`, optional
        A seed for the random number generator.
    weighted: `bool`, optional
        Whether the edge weights should be uniform or different. If false, all weights are set to 1.
        If true, the weight is set to a random number drawn from the uniform distribution in the
        interval 0 to 1.
    biases: `bool`, optional
        Whether or not the graph nodes should be assigned a weight.
        If true, the weight is set to a random number drawn from the uniform
        distribution in the interval 0 to 1.

    Returns
    -------
    nx.Graph: `Networkx Graph`
        A graph with the properties as specified.
    """
    # Set numpy seed
    np.random.seed(seed=seed)

    # Create a random regular graph on the nodes
    G = nx.random_regular_graph(degree, len(nodes), seed)

    # Relabel nodes
    nx.relabel_nodes(G, {i: n for i, n in enumerate(nodes)})

    # Add edges between nodes
    for edge in G.edges():

        # If weighted attribute is False, all weights are set to 1
        if not weighted:
            G[edge[0]][edge[1]]['weight'] = 1

        # If weighted attribute is True, weights are assigned as random integers
        else:
            G[edge[0]][edge[1]]['weight'] = np.random.normal(mu, sigma)

    # If biases attribute is True, add node weights as random integers
    if biases:
        for node in G.nodes():
            G.nodes[node]['weight'] = np.random.rand()

    return G

In [4]:
# helper function to produce the cost landscape
def create_cost_landscape_std(hamiltonian, betas, gammas):
    """
    This function constructs a 2-D array containing cost values for different pairs
    of parameter values.
    
    """
    
    cost_landscape = np.zeros((betas.size, gammas.size), dtype=float)
    
    for i, beta in enumerate(betas):
        for j, gamma in enumerate(gammas):
            variational_params = [beta, gamma]
            cost_landscape[i,j] = energy_expectation_analytical(variational_params, hamiltonian)
            
           
    return cost_landscape

In [5]:
def plot_landscape(cost_landscape, betas, gammas, sigma, mu):

    plt.contourf(cost_landscape, cmap='viridis', levels=100, extent=(gammas[0],gammas[-1],betas[0],betas[-1]))

    #plt.title(r'Cost Landscape for Standard Parameterization for $\sigma$={}'.format(sigma))
    plt.xlabel(r'$\gamma$')
    plt.ylabel(r'$\beta$')
    plt.colorbar()
    #plt.savefig("cost_landscape_sigma={}_mu={}.png".format(sigma, mu))
    plt.show()
    

In [6]:
def plot_landscape_with_opt_trajectory(cost_landscape, optimization_angles, betas, gammas, optimizer):
    plt.contourf(cost_landscape, cmap='viridis', levels=100, extent=(gammas[0],gammas[-1],betas[0],betas[-1]))
        
    plt.plot(optimization_angles[:, 1], optimization_angles[:, 0], '--', color='black',  )  
    plt.plot(optimization_angles[0, 1], optimization_angles[0, 0], 'x', color='black',  ) 
    
    plt.plot(optimization_angles[-1, 1], optimization_angles[-1, 0], 'x', color='orange',  )

    #plt.title(r'Cost Landscape for Standard Parameterization for $\sigma$={}'.format(sigma))
    plt.xlabel(r'$\gamma$')
    plt.ylabel(r'$\beta$')
    plt.colorbar()
    plt.savefig("cost_landscape_trajectory_optimizer={}.png".format(optimizer))
    plt.show()

### Analyze experiment with initializations

In [ ]:
experiment_name = 'Maxcut_w3R_initializations_smallest_region' # to be used as a name for the folder containing all jsons for this experiment 
# structure of the name: problem_typeOfGraph[optional]_variableToInvestigate

mu = 5.0

sigma_list = [10]
#optimizer_list = ["vgd", "rmsprop", 'newton-cg', 'cobyla', 'nelder-mead', 'powell' ]
optimizer_list = ["cobyla"]
graph_seed_list = [23]
init_seed_list = [8000]


data = []

for sigma in sigma_list:
    print("Sigma=", sigma) 
    for graph_seed in graph_seed_list:
        print("graph_seed=", graph_seed)
        for optimizer in optimizer_list:
            print("Optimizer=", optimizer)
            for init_seed in init_seed_list:
                print("init_seed=", init_seed)
                
                # Opening JSON file
                with open("{}/mu_{}__sigma_{}__graphSeed_{}_optimizer_{}__initSeed_{}.json".format(experiment_name, mu, sigma, graph_seed, optimizer, init_seed)) as f:

                    # returns JSON object as a dictionary
                    opt_results = json.load(f)

                    # construct the original problem Hamiltonian from the initial QAOA 
                    qaoa_init = opt_results['intermediate_steps'][0]['QAOA']
                    terms = []
                    for i in range(0, len(qaoa_init['cost_hamiltonian']['terms'])):
                        terms.append(qaoa_init['cost_hamiltonian']['terms'][i]['qubit_indices'])
                    weights = qaoa_init['cost_hamiltonian']['coeffs']
                    const = qaoa_init['cost_hamiltonian']['constant']

                    cost_hamiltonian = Hamiltonian.classical_hamiltonian(terms, weights, const)

                    analytical_energy, configuration = ground_state_hamiltonian(cost_hamiltonian)

                    # Calculate the approx. ratio
                    optimization_energy = list(opt_results['solution'].values())[0]

                    r = np.abs(optimization_energy) / np.abs(analytical_energy)

                    # Extract other params
                    optimization_schedule = list(opt_results['schedule'])

                    optimization_angles = np.array(opt_results['intermediate_steps'][3]['QAOA']['intermediate']['angles log'])
    
                    data.append([sigma, graph_seed, init_seed, optimizer, r])
        



In [ ]:
betas = np.linspace(0, np.pi, 100)
gammas = np.linspace(0, np.pi, 100)

cost_landscape_std = create_cost_landscape_std(cost_hamiltonian, betas, gammas)

In [ ]:
plot_landscape(cost_landscape_std, betas, gammas, sigma, mu)


In [ ]:
plot_landscape(cost_landscape_std, betas, gammas, sigma, mu)


In [ ]:
plot_landscape_with_opt_trajectory(cost_landscape_std, optimization_angles, betas, gammas, optimizer)

# Analyze experiment with grid search and different types of RQAOA

In [7]:
experiment_name = 'Maxcut_w3R_grid_search' # to be used as a name for the folder containing all jsons for this experiment 
# structure of the name: problem_typeOfGraph[optional]_variableToInvestigate

mu = 5.0

sigma_list = [0.1]
#optimizer_list = ["vgd", "rmsprop", 'newton-cg', 'cobyla', 'nelder-mead', 'powell' ]
optimizer_list = ["rmsprop"]
graph_seed_list = [5]
rqaoa_type_list = ["custom"]


data = []

for sigma in sigma_list:
    print("Sigma=", sigma) 
    for graph_seed in graph_seed_list:
        print("graph_seed=", graph_seed)
        for optimizer in optimizer_list:
            print("Optimizer=", optimizer)
            for rqaoa_type in rqaoa_type_list:
                print("rqaoa_type=", rqaoa_type)
                
                # Opening JSON file
                with open("{}/mu_{}__sigma_{}__graphSeed_{}_optimizer_{}__type_{}__steps_2.json".format(experiment_name, mu, sigma, graph_seed, optimizer, rqaoa_type)) as f:

                    # returns JSON object as a dictionary
                    opt_results = json.load(f)

                    # construct the original problem Hamiltonian from the initial QAOA 
                    qaoa_init = opt_results['intermediate_steps'][0]['QAOA']
                    terms = []
                    for i in range(0, len(qaoa_init['cost_hamiltonian']['terms'])):
                        terms.append(qaoa_init['cost_hamiltonian']['terms'][i]['qubit_indices'])
                    weights = qaoa_init['cost_hamiltonian']['coeffs']
                    const = qaoa_init['cost_hamiltonian']['constant']

                    cost_hamiltonian = Hamiltonian.classical_hamiltonian(terms, weights, const)

                    analytical_energy, configuration = ground_state_hamiltonian(cost_hamiltonian)

                    # Calculate the approx. ratio
                    optimization_energy = list(opt_results['solution'].values())[0]

                    r = np.abs(optimization_energy) / np.abs(analytical_energy)

                    # Extract other params
                    optimization_schedule = list(opt_results['schedule'])

                    optimization_angles = np.array(opt_results['intermediate_steps'][0]['QAOA']['intermediate']['angles log'])
    
                    data.append([sigma, graph_seed, rqaoa_type, optimizer, r])
        



Sigma= 0.1
graph_seed= 5
Optimizer= rmsprop
rqaoa_type= custom


In [10]:
intermediate_steps

NameError: name 'intermediate_steps' is not defined

In [15]:
opt_results['intermediate_steps'][0]['QAOA']['intermediate']['angles log']

[[0.38468481472528077, 0.25216797219449844],
 [0.38468481472528077, 0.25216797219449844],
 [0.3836848148176044, 0.2531679721895679],
 [0.3832698549732316, 0.2531803736712282],
 [0.38306719788802146, 0.25317754363997313],
 [0.38297304633812607, 0.2531766552630488],
 [0.3829323127909203, 0.25317622505004883],
 [0.3829160502026685, 0.2531760614491291]]

In [18]:
betas = np.linspace(0, np.pi/2, 50)
gammas = np.linspace(0, 3*np.pi, 300)

intermediate_steps = len(opt_results['intermediate_steps'])

for i in range(0, intermediate_steps):
    print(i)
    intermediate_qubo = opt_results['intermediate_steps'][i]['QUBO']
    
    terms = []
    for i in range(0, len(intermediate_qubo['cost_hamiltonian']['terms'])):
        terms.append(intermediate_qubo['cost_hamiltonian']['terms'][i]['qubit_indices'])
    weights = intermediate_qubo['cost_hamiltonian']['coeffs']
    const = intermediate_qubo['cost_hamiltonian']['constant']

    intermediate_hamiltonian = Hamiltonian.classical_hamiltonian(terms, weights, const)

    intermediate_g = graph_from_hamiltonian(intermediate_hamiltonian)
    optimization_angles = np.array(opt_results['intermediate_steps'][0]['QAOA']['intermediate']['angles log'])
    print(intermediate_g)
    plot_graph(intermediate_g)
    plt.clf()
    cost_landscape = create_cost_landscape_std(intermediate_hamiltonian, betas, gammas)
    #plot_landscape(cost_landscape, betas, gammas, sigma, mu)
    plot_landscape_with_opt_trajectory(cost_landscape, optimization_angles, betas, gammas, optimizer)



0


KeyError: 'cost_hamiltonian'

In [ ]:
plot_landscape_with_opt_trajectory(cost_landscape_std, optimization_angles, betas, gammas, optimizer)